# 解析错误时如何重试

虽然在某些情况下，仅通过查看输出来修复任何解析错误是可能的，但在其他情况下则不行。一个例子是，当输出不仅格式不正确，而且还不完整。请看下面的例子。

In [1]:
from langchain.output_parsers import OutputFixingParser
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAI
from pydantic import BaseModel, Field

In [2]:
template = """Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:"""


class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")


parser = PydanticOutputParser(pydantic_object=Action)

In [3]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [4]:
prompt_value = prompt.format_prompt(query="who is leo di caprios gf?")

In [5]:
bad_response = '{"action": "search"}'

如果我们尝试按原样解析此响应，我们将收到一个错误：

In [6]:
try:
    parser.parse(bad_response)
except OutputParserException as e:
    print(e)

Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
action_input
  Field required [type=missing, input_value={'action': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE


如果我们尝试使用 `OutputFixingParser` 来修复此错误，它将感到困惑——具体来说，它不知道该为 action input 实际输入什么。

In [7]:
fix_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [8]:
fix_parser.parse(bad_response)

Action(action='search', action_input='input')

不过，我们可以使用 `RetryOutputParser`，它会将 prompt（以及原始输出）传入，以再次尝试获得更好的响应。

In [9]:
from langchain.output_parsers import RetryOutputParser

In [10]:
retry_parser = RetryOutputParser.from_llm(parser=parser, llm=OpenAI(temperature=0))

In [11]:
retry_parser.parse_with_prompt(bad_response, prompt_value)

Action(action='search', action_input='leo di caprio girlfriend')

我们还可以轻松地为自定义链添加 RetryOutputParser，该链可将原始 LLM/ChatModel 输出转换为更易于处理的格式。

In [1]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

completion_chain = prompt | OpenAI(temperature=0)

main_chain = RunnableParallel(
    completion=completion_chain, prompt_value=prompt
) | RunnableLambda(lambda x: retry_parser.parse_with_prompt(**x))


main_chain.invoke({"query": "who is leo di caprios gf?"})

Action(action='search', action_input='leo di caprio girlfriend')


查找 [RetryOutputParser](https://python.langchain.com/api_reference/langchain/output_parsers/langchain.output_parsers.retry.RetryOutputParser.html#langchain.output_parsers.retry.RetryOutputParser) 的 API 文档。